In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import h5py
import copy
import datetime
#import ta
import cProfile
#import yfinance as yf
#import tensorflow as tf
#import tensorflow.compat.v2.feature_column as fc
from IPython.display import clear_output
import pyodbc

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, log_loss

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning 
from sklearn import ensemble
# ConvergenceWarning('ignore')
# Do you wanna see?
verbose = True

import sys
sys.path.append('../')

from utils.data_extraction import load_data
from utils.data_cleaning import HFDataCleaning
from utils.generate_features import candleCreateNP_vect_final,\
                                    generateFeatures_final
from utils.preprocessing_features_and_labels import extract_labels,\
                                                    align_features_and_labels,\
                                                    pre_processing_initial,\
                                                    pre_processing_extended,\
                                                    pre_processing
#from utils.models import make_input_fn
#from utils.models import performanceTesting,scoreFunction

In [2]:
#!pip install ta

## Let's extract some data now!

In [12]:
# print(os.listdir())
try:
    path = 'a:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)
except:
    path = 'f:/taqhdf5' # 't:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)    
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

#dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,16)]).astype(int)
dates = np.array(['20200501', '20200504']).astype(int)#,'20200402','20200403','20200406','20200407'

# Provide a list of tickers of interest
tickers = ['GOOG','TSLA']#'MSFT'

# Do we need data on trades, quotes or both?
dataNeeded = 'quotes' # 'trades', 'quotes' or 'both'

# run load_data()
# if dataNeeded == 'trades':
#     tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'quotes':
#     quoteData = load_data(dates, tickers, dataNeeded, path, False)
# elif dataNeeded == 'both':
#     tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

In [8]:
quoteData

NameError: name 'quoteData' is not defined

## Optimizing the data loader

In [11]:
# Function to clean the unpacked data from the compressed files.
def strList(ls):
    return list(map(lambda x: x.decode('utf-8'),ls))

# The following function is based on the research of (Lunde, 2016), summarized in the slides found here:
# https://econ.au.dk/fileadmin/site_files/filer_oekonomi/subsites/creates/Diverse_2016/PhD_High-Frequency/HF_TrQuData_v01.pdf

def formatDate(date,timestamps):
    return list(map(lambda x: date[0:4]+'/'+date[4:6]+'/'+date[6:]+' '+str(datetime.timedelta(seconds = int(str(x)[0:5]),
                                                     microseconds = int(str(x)[5:11]))),timestamps))


def load_data_v1(dates, tickers, dataNeeded, path, verbose):
    # Measuring the exraction time
    start = time.time()
    
    allFiles = os.listdir(path)

    # Extracting just the dates of each file
    allDates = np.array([re.split("[._]",ele)[1] if ("." in ele ) & ("_" in ele) else 0 for ele in allFiles]).astype(int)

    minDate = np.min(dates)
    maxDate = np.max(dates)
    
    print('1 Lap time: %.3f' % ((time.time()-start)))

    # Locating what files we need.
    index = np.where((minDate <= allDates) & (allDates <= maxDate))
    
    relevantFiles = np.array(allFiles)[index[0]]
    
    print('2 Lap time: %.3f' % ((time.time()-start)))
    
    # Separating the files into trade and quote files.
#     trade = [ele for ele in relevantFiles if 'trade' in ele]
    quote = [ele for ele in relevantFiles if 'quote' in ele]

    if (dataNeeded == 'both') | (dataNeeded == 'quotes'):

        # Now to the quote data
        for i,file in enumerate(quote):

#             if (verbose) & (i == 0):
#                 print('### Quote Data ###\n')

            # Reading one file at a time
            raw_data = h5py.File(path+'/'+file,'r')
            dt2 = raw_data['Quotes'].dtype
            print('3 Lap time: %.3f' % ((time.time()-start)))
            
            
            # Store the trade indecies
            QI = raw_data['QuoteIndex']
            
            print('4 Lap time: %.3f' % ((time.time()-start)))
#             if (verbose) & (i==0):
#                 print('The raw H5 quote file contains: ',list(raw_data.keys()),'\n')

            # Extracting just the tickers
            QIC = np.array([ele[0].astype(str).strip() for ele in QI])
            
            print('5 Lap time: %.3f' % ((time.time()-start)))
            
            # Lets get data on each ticker for the file processed at the moment
            for j,ticker in enumerate(tickers):
                print(ticker)
                #print(ticker in QIC)
                # Getting the specific ticker information
#                 print(np.where(ticker==QIC))
#                 print(QI[5255])
                #print(QI[QIC==ticker])
                
                tickerInfo = QI[QIC==ticker][0]
                #print(tickerInfo)
                print('6 Lap time: %.3f' % ((time.time()-start)))
                
                
#                 if (verbose) & (i == 0):
#                         print('Ticker Information: ',tickerInfo,'\n')

                # Raw data
#                 tempData = raw_data['Quotes'][list(np.arange(tickerInfo[1],tickerInfo[1]+tickerInfo[2]))]
#                 print(tickerInfo[1],tickerInfo[2])
#                 print('Datatype: ',type(raw_data['Quotes']))
#                 print(raw_data['Quotes'].dtype)
#                 tempData = raw_data['Quotes'][tickerInfo[1]:(tickerInfo[1]+tickerInfo[2])]
#                 tempData = raw_data['Quotes'][403024728:600343]
#                 print(raw_data['Quotes'][[0,1,2,3,4,5]])
#                 print(raw_data['Quotes'][0:5])

                # Read direct
#                 print(raw_data['Quotes'][0])
#                 ll = len(raw_data['Quotes'][0])
#                 print(type(raw_data['Quotes']))
#                 print(type(raw_data['Quotes'][0]))
#                 arr = np.zeros((10,))#,dtype=object,(tickerInfo[2],), dtype='int32'
#                 print(arr)
# #                 print(np.s_[0:10])
#                 raw_data['Quotes'].read_direct(arr,
#                                                np.s_[0:10],#,tickerInfo[1]:tickerInfo[1]+tickerInfo[2]
#                                                np.s_[0:10])#tickerInfo[2]
#                 print(arr[0:5])
        
                ## New proposal
            
#                 store = pd.HDFStore(path+'/'+file, 'r') #.get('Quotes')#'taqquote_20200401.h5'
#                 print(store is None)
#                 meta = store.select('Quotes', start=1, stop=1)
#                 dtDict = [(i,j) for i,j in zip(meta.columns,meta.dtypes)]
#                 dt2 = np.dtype(dtDict)
                
                #testDset = raw_data['Quotes']
                #test = testDset[testDset[0]]
#                 print(dt2)
                arr = np.zeros((tickerInfo[2].astype(int)+1,),dtype=dt2)#,dtype=object,(tickerInfo[2],), dtype='int32'
#                 tickerInfo[1] = 0
#                 tickerInfo[2] = 500000
#                 print(np.s_[0:10])
                #print(int(tickerInfo[1]+tickerInfo[2]+1))
                #print(tickerInfo[1])
                #print(tickerInfo[1]+tickerInfo[2])
                raw_data['Quotes'].read_direct(arr,
                                               np.s_[tickerInfo[1]:int(tickerInfo[1]+tickerInfo[2]+1)],#,tickerInfo[1]:tickerInfo[1]+tickerInfo[2]
                                               np.s_[0:int(tickerInfo[2]+1)])#tickerInfo[2]
#                 print(arr)
#                 tempData = pd.DataFrame(arr,columns=dt2.names)
                print('7 Lap time: %.3f' % ((time.time()-start)))
                
                # For first file and first ticker.
                if (i == 0) & (j == 0):

#                     quoteData = pd.DataFrame(tempData, columns= tempData.dtype.names)
                    quoteData = pd.DataFrame(arr, columns= dt2.names)
                    # We remove all unnecessary variables
                    unnecessaryVariables = ['NationalBBOInd',
                                            'FinraBBOInd',
                                            'FinraQuoteIndicator',
                                            'SequenceNumber',
                                            'FinraAdfMpidIndicator',
                                            'QuoteCancelCorrection',
                                            'SourceQuote',
                                            'RPI',
                                            'ShortSaleRestrictionIndicator',
                                            'LuldBBOIndicator',
                                            'SIPGeneratedMessageIdent',
                                            'NationalBBOLuldIndicator',
                                            'ParticipantTimestamp',
                                            'FinraTimestamp',
                                            'FinraQuoteIndicator',
                                            'SecurityStatusIndicator']

                    quoteData = quoteData.drop(columns=unnecessaryVariables)
                    
                    print('8 Lap time: %.3f' % ((time.time()-start)))
                    
                    
                    quoteData.loc[:,'ex'] = strList(quoteData.ex)
                    print('9 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData.loc[:,'mode'] = strList(quoteData['mode'])
                    print('10 Lap time: %.3f' % ((time.time()-start)))
                    
                    # Adding the date of the file to the dataframe.
                    quoteData['Date'] = re.split('[._]',
                                                 file)[1]
                    print('11 Lap time: %.3f' % ((time.time()-start)))
                    
                    # Adding a more readable timestamp - TEST IT
                    quoteData['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',
                                                                                file)[1],
                                                                       quoteData.utcsec))
                    print('12 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData['TSRemainder'] = list(map(lambda x: str(x)[11:],
                                                        quoteData.utcsec))
                    
                    print('13 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData['Hour'] = quoteData.Timestamp.dt.hour
                    quoteData['Minute'] = quoteData.Timestamp.dt.minute
                    # Adding the ticker
                    quoteData['Ticker'] = ticker

                    if (verbose) & (i==0) & (j==0):
                        print('Sneak peak of the data\n\n',quoteData.head())
                    
                    #raw_data.close()
                else:

                    # Storing the data on the following tickers in a temporary variable.

                    temp = pd.DataFrame(arr, columns= dt2.names)
                    # Removing all unnecessary variables
                    temp = temp.drop(columns=unnecessaryVariables)

                    temp.loc[:,'ex'] = strList(temp.ex)
                    temp.loc[:,'mode'] = strList(temp['mode'])

                    # Adding the date of the file to the dataframe.
                    temp['Date'] = re.split('[._]',file)[1]

                    # Adding a more readable timestamp - TEST IT
                    temp['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',file)[1],temp.utcsec))
                    temp['TSRemainder'] = list(map(lambda x: str(x)[11:], temp.utcsec))
                    temp['Hour'] = temp.Timestamp.dt.hour
                    temp['Minute'] = temp.Timestamp.dt.minute

                    # Adding the ticker
                    temp['Ticker'] = ticker

                    # Adding the new data
                    quoteData = pd.concat([quoteData,temp])

    end = time.time()

    print('The extraction time was %.3f seconds.' % (end-start))

    quoteData.loc[:,'price'] = (quoteData.bid + quoteData.ofr) / 2
    quoteData.loc[:,'spread'] = quoteData.ofr - quoteData.bid
    return quoteData

def load_data_v2(dates, tickers, dataNeeded, path, verbose):
    # Measuring the exraction time
    start = time.time()
    
    allFiles = os.listdir(path)

    # Extracting just the dates of each file
    allDates = np.array([re.split("[._]",ele)[1] if ("." in ele ) & ("_" in ele) else 0 for ele in allFiles]).astype(int)

    minDate = np.min(dates)
    maxDate = np.max(dates)
    
    print('1 Lap time: %.3f' % ((time.time()-start)))

    # Locating what files we need.
    index = np.where((minDate <= allDates) & (allDates <= maxDate))
    
    relevantFiles = np.array(allFiles)[index[0]]
    
    print('2 Lap time: %.3f' % ((time.time()-start)))
    
    # Separating the files into trade and quote files.
#     trade = [ele for ele in relevantFiles if 'trade' in ele]
    quote = [ele for ele in relevantFiles if 'quote' in ele]
    
    if (dataNeeded == 'both') | (dataNeeded == 'quotes'):

        # Now to the quote data
        for i,file in enumerate(quote):

#             if (verbose) & (i == 0):
#                 print('### Quote Data ###\n')

            # Reading one file at a time
            raw_data = h5py.File(path+'/'+file,'r')
            dt2 = raw_data['Quotes'].dtype
            print('3 Lap time: %.3f' % ((time.time()-start)))
            
            
            # Store the trade indecies
            QI = raw_data['QuoteIndex']
            
            print('4 Lap time: %.3f' % ((time.time()-start)))
#             if (verbose) & (i==0):
#                 print('The raw H5 quote file contains: ',list(raw_data.keys()),'\n')

            # Extracting just the tickers
            QIC = np.array([ele[0].astype(str).strip() for ele in QI])
            
            print('5 Lap time: %.3f' % ((time.time()-start)))
            pos_start = []
            pos_range = []
            # Lets get data on each ticker for the file processed at the moment
            for j,ticker in enumerate(tickers):
                print(ticker)
                
                tickerInfo = QI[QIC==ticker][0]
                pos_start.append(tickerInfo[1])
                pos_range.append(tickerInfo[2])
                print(tickerInfo)
            print('6 Lap time: %.3f' % ((time.time()-start)))

            # Raw data
            data_range = np.concatenate([np.arange(i,j+i) for i,j in zip(pos_start,pos_range)])
            tempData = raw_data['Quotes'][data_range]
                       
            print('7 Lap time: %.3f' % ((time.time()-start)))

            # For first file and first ticker.
            if (i == 0):

                quoteData = pd.DataFrame(tempData, columns= dt2.names)
                # We remove all unnecessary variables
                unnecessaryVariables = ['NationalBBOInd',
                                        'FinraBBOInd',
                                        'FinraQuoteIndicator',
                                        'SequenceNumber',
                                        'FinraAdfMpidIndicator',
                                        'QuoteCancelCorrection',
                                        'SourceQuote',
                                        'RPI',
                                        'ShortSaleRestrictionIndicator',
                                        'LuldBBOIndicator',
                                        'SIPGeneratedMessageIdent',
                                        'NationalBBOLuldIndicator',
                                        'ParticipantTimestamp',
                                        'FinraTimestamp',
                                        'FinraQuoteIndicator',
                                        'SecurityStatusIndicator']

                quoteData = quoteData.drop(columns=unnecessaryVariables)

                print('8 Lap time: %.3f' % ((time.time()-start)))


                quoteData.loc[:,'ex'] = strList(quoteData.ex)
                print('9 Lap time: %.3f' % ((time.time()-start)))

                quoteData.loc[:,'mode'] = strList(quoteData['mode'])
                print('10 Lap time: %.3f' % ((time.time()-start)))

                # Adding the date of the file to the dataframe.
                quoteData['Date'] = re.split('[._]',
                                             file)[1]
                print('11 Lap time: %.3f' % ((time.time()-start)))

                # Adding a more readable timestamp - TEST IT
                dates = pd.to_datetime(quoteData.loc[:,'Date'], format='%Y%m%d', errors='ignore')
                times = pd.to_timedelta(quoteData.loc[:,'utcsec'])
#                 datetimes  = dates + times
#                 datetimes
                quoteData['Timestamp'] = dates + times
#                 quoteData['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',
#                                                                             file)[1],
#                                                                    quoteData.utcsec))
                print('12 Lap time: %.3f' % ((time.time()-start)))

#                 quoteData['TSRemainder'] = list(map(lambda x: str(x)[11:],
#                                                     quoteData.utcsec))

                print('13 Lap time: %.3f' % ((time.time()-start)))

                quoteData['Hour'] = quoteData.Timestamp.dt.hour
                quoteData['Minute'] = quoteData.Timestamp.dt.minute
                # Adding the ticker
                
            else:

                # Storing the data on the following tickers in a temporary variable.

                temp = pd.DataFrame(tempData, columns= d2.names)
                # Removing all unnecessary variables
                temp = temp.drop(columns=unnecessaryVariables)
                
                print('8 Lap time: %.3f' % ((time.time()-start)))

                temp.loc[:,'ex'] = strList(temp.ex)
                
                print('9 Lap time: %.3f' % ((time.time()-start)))
                
                temp.loc[:,'mode'] = strList(temp['mode'])
                
                print('10 Lap time: %.3f' % ((time.time()-start)))
                
                # Adding the date of the file to the dataframe.
                temp['Date'] = re.split('[._]',file)[1]
                
                print('11 Lap time: %.3f' % ((time.time()-start)))
                
                # Adding a more readable timestamp - TEST IT
                temp['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',file)[1],temp.utcsec))
                
                print('12 Lap time: %.3f' % ((time.time()-start)))
                
                temp['TSRemainder'] = list(map(lambda x: str(x)[11:], temp.utcsec))
                temp['Hour'] = temp.Timestamp.dt.hour
                temp['Minute'] = temp.Timestamp.dt.minute

                # Adding the ticker
                temp['Ticker'] = ticker

                # Adding the new data
                quoteData = pd.concat([quoteData,temp])
            
            # Closing the file after having used it.
            raw_data.close()
    end = time.time()

    print('The extraction time was %.3f seconds.' % (end-start))

    quoteData.loc[:,'price'] = (quoteData.bid + quoteData.ofr) / 2
    quoteData.loc[:,'spread'] = quoteData.ofr - quoteData.bid
    return quoteData

def load_data_v3(dates, tickers, dataNeeded, path, verbose):
    # Measuring the exraction time
    start = time.time()
    
    allFiles = os.listdir(path)

    # Extracting just the dates of each file
    allDates = np.array([re.split("[._]",ele)[1] if ("." in ele ) & ("_" in ele) else 0 for ele in allFiles]).astype(int)

    minDate = np.min(dates)
    maxDate = np.max(dates)
    
    print('1 Lap time: %.3f' % ((time.time()-start)))

    # Locating what files we need.
    index = np.where((minDate <= allDates) & (allDates <= maxDate))
    
    relevantFiles = np.array(allFiles)[index[0]]
    
    print('2 Lap time: %.3f' % ((time.time()-start)))
    
    # Separating the files into trade and quote files.
#     trade = [ele for ele in relevantFiles if 'trade' in ele]
    quote = [ele for ele in relevantFiles if 'quote' in ele]

    if (dataNeeded == 'both') | (dataNeeded == 'quotes'):

        # Now to the quote data
        for i,file in enumerate(quote):

#             if (verbose) & (i == 0):
#                 print('### Quote Data ###\n')

            # Reading one file at a time
            raw_data = h5py.File(path+'/'+file,'r')
            dt2 = raw_data['Quotes'].dtype
            print('3 Lap time: %.3f' % ((time.time()-start)))
            
            
            # Store the trade indecies
            QI = raw_data['QuoteIndex']
            
            print('4 Lap time: %.3f' % ((time.time()-start)))
#             if (verbose) & (i==0):
#                 print('The raw H5 quote file contains: ',list(raw_data.keys()),'\n')

            # Extracting just the tickers
            QIC = np.array([ele[0].astype(str).strip() for ele in QI])
            
            print('5 Lap time: %.3f' % ((time.time()-start)))
            
            store = pd.HDFStore(path+'/'+file, 'r')
            
            # Lets get data on each ticker for the file processed at the moment
            for j,ticker in enumerate(tickers):
                print(ticker)
                
                tickerInfo = QI[QIC==ticker][0]

                print('6 Lap time: %.3f' % ((time.time()-start)))
                
                meta = store.select('Quotes', start=tickerInfo[1], stop=(tickerInfo[1]+tickerInfo[2]))

                print('7 Lap time: %.3f' % ((time.time()-start)))
                
                # For first file and first ticker.
                if (i == 0) & (j == 0):

#                     quoteData = pd.DataFrame(tempData, columns= tempData.dtype.names)
                    quoteData = meta.copy(deep=True)#pd.DataFrame(arr, columns= dt2.names)
                    # We remove all unnecessary variables
                    unnecessaryVariables = ['NationalBBOInd',
                                            'FinraBBOInd',
                                            'FinraQuoteIndicator',
                                            'SequenceNumber',
                                            'FinraAdfMpidIndicator',
                                            'QuoteCancelCorrection',
                                            'SourceQuote',
                                            'RPI',
                                            'ShortSaleRestrictionIndicator',
                                            'LuldBBOIndicator',
                                            'SIPGeneratedMessageIdent',
                                            'NationalBBOLuldIndicator',
                                            'ParticipantTimestamp',
                                            'FinraTimestamp',
                                            'FinraQuoteIndicator',
                                            'SecurityStatusIndicator']

                    quoteData = quoteData.drop(columns=unnecessaryVariables)
                    
                    print('8 Lap time: %.3f' % ((time.time()-start)))
                    
                    
#                     quoteData.loc[:,'ex'] = strList(quoteData.ex)
                    print('9 Lap time: %.3f' % ((time.time()-start)))
                    
#                     quoteData.loc[:,'mode'] = strList(quoteData['mode'])
                    print('10 Lap time: %.3f' % ((time.time()-start)))
                    
                    # Adding the date of the file to the dataframe.
                    quoteData['Date'] = re.split('[._]',
                                                 file)[1]
                    print('11 Lap time: %.3f' % ((time.time()-start)))
                    
                    # Adding a more readable timestamp - TEST IT
                    quoteData['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',
                                                                                file)[1],
                                                                       quoteData.utcsec))
                    print('12 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData['TSRemainder'] = list(map(lambda x: str(x)[11:],
                                                        quoteData.utcsec))
                    
                    print('13 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData['Hour'] = quoteData.Timestamp.dt.hour
                    quoteData['Minute'] = quoteData.Timestamp.dt.minute
                    # Adding the ticker
                    quoteData['Ticker'] = ticker

                    if (verbose) & (i==0) & (j==0):
                        print('Sneak peak of the data\n\n',quoteData.head())
                    
                    #raw_data.close()
                else:

                    # Storing the data on the following tickers in a temporary variable.

                    temp = meta.copy(deep=True)#pd.DataFrame(arr, columns= dt2.names)
                    # Removing all unnecessary variables
                    temp = temp.drop(columns=unnecessaryVariables)

#                     temp.loc[:,'ex'] = strList(temp.ex)
#                     temp.loc[:,'mode'] = strList(temp['mode'])

                    # Adding the date of the file to the dataframe.
                    temp['Date'] = re.split('[._]',file)[1]

                    # Adding a more readable timestamp - TEST IT
                    temp['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',file)[1],temp.utcsec))
                    temp['TSRemainder'] = list(map(lambda x: str(x)[11:], temp.utcsec))
                    temp['Hour'] = temp.Timestamp.dt.hour
                    temp['Minute'] = temp.Timestamp.dt.minute

                    # Adding the ticker
                    temp['Ticker'] = ticker

                    # Adding the new data
                    quoteData = pd.concat([quoteData,temp])

    end = time.time()

    print('The extraction time was %.3f seconds.' % (end-start))

    quoteData.loc[:,'price'] = (quoteData.bid + quoteData.ofr) / 2
    quoteData.loc[:,'spread'] = quoteData.ofr - quoteData.bid
    return quoteData

def load_data_v4(dates, tickers, dataNeeded, path, verbose):
    # Measuring the exraction time
    start = time.time()
    
    allFiles = os.listdir(path)

    # Extracting just the dates of each file
    allDates = np.array([re.split("[._]",ele)[1] if ("." in ele ) & ("_" in ele) else 0 for ele in allFiles]).astype(int)

    minDate = np.min(dates)
    maxDate = np.max(dates)
    
    print('1 Lap time: %.3f' % ((time.time()-start)))

    # Locating what files we need.
    index = np.where((minDate <= allDates) & (allDates <= maxDate))
    
    relevantFiles = np.array(allFiles)[index[0]]
    
    print('2 Lap time: %.3f' % ((time.time()-start)))
    
    # Separating the files into trade and quote files.
#     trade = [ele for ele in relevantFiles if 'trade' in ele]
    quote = [ele for ele in relevantFiles if 'quote' in ele]
    
    if (dataNeeded == 'both') | (dataNeeded == 'quotes'):

        # Now to the quote data
        for i,file in enumerate(quote):

#             if (verbose) & (i == 0):
#                 print('### Quote Data ###\n')

            # Reading one file at a time
            raw_data = h5py.File(path+'/'+file,'r')
            dt2 = raw_data['Quotes'].dtype
            print('3 Lap time: %.3f' % ((time.time()-start)))
            
            
            # Store the trade indecies
            QI = raw_data['QuoteIndex']
            
            print('4 Lap time: %.3f' % ((time.time()-start)))
#             if (verbose) & (i==0):
#                 print('The raw H5 quote file contains: ',list(raw_data.keys()),'\n')

            # Extracting just the tickers
            QIC = np.array([ele[0].astype(str).strip() for ele in QI])
            
            print('5 Lap time: %.3f' % ((time.time()-start)))
            pos_start = []
            pos_range = []
            # Lets get data on each ticker for the file processed at the moment
            for j,ticker in enumerate(tickers):
                print(ticker)
                
                tickerInfo = QI[QIC==ticker][0]
                pos_start.append(tickerInfo[1])
                pos_range.append(tickerInfo[2])
                print(tickerInfo)
            print('6 Lap time: %.3f' % ((time.time()-start)))

                # Raw data
            data_range = np.concatenate([np.arange(i,j+i) for i,j in zip(pos_start,pos_range)])
            tempData = raw_data['Quotes'][np.r_[data_range]]
            l = len(data_range)
            arr = np.zeros((l,),dtype=dt2)

            raw_data['Quotes'].read_direct(arr,
                                           np.s_[data_range],#,tickerInfo[1]:tickerInfo[1]+tickerInfo[2]
                                           np.s_[0:l])#tickerInfo[2]
            
            print('7 Lap time: %.3f' % ((time.time()-start)))

            # For first file and first ticker.
            if (i == 0):

                quoteData = pd.DataFrame(tempData, columns= dt2.names)
                # We remove all unnecessary variables
                unnecessaryVariables = ['NationalBBOInd',
                                        'FinraBBOInd',
                                        'FinraQuoteIndicator',
                                        'SequenceNumber',
                                        'FinraAdfMpidIndicator',
                                        'QuoteCancelCorrection',
                                        'SourceQuote',
                                        'RPI',
                                        'ShortSaleRestrictionIndicator',
                                        'LuldBBOIndicator',
                                        'SIPGeneratedMessageIdent',
                                        'NationalBBOLuldIndicator',
                                        'ParticipantTimestamp',
                                        'FinraTimestamp',
                                        'FinraQuoteIndicator',
                                        'SecurityStatusIndicator']

                quoteData = quoteData.drop(columns=unnecessaryVariables)

                print('8 Lap time: %.3f' % ((time.time()-start)))


                quoteData.loc[:,'ex'] = strList(quoteData.ex)
                print('9 Lap time: %.3f' % ((time.time()-start)))

                quoteData.loc[:,'mode'] = strList(quoteData['mode'])
                print('10 Lap time: %.3f' % ((time.time()-start)))

                # Adding the date of the file to the dataframe.
                quoteData['Date'] = re.split('[._]',
                                             file)[1]
                print('11 Lap time: %.3f' % ((time.time()-start)))

                # Adding a more readable timestamp - TEST IT
                quoteData['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',
                                                                            file)[1],
                                                                   quoteData.utcsec))
                print('12 Lap time: %.3f' % ((time.time()-start)))

                quoteData['TSRemainder'] = list(map(lambda x: str(x)[11:],
                                                    quoteData.utcsec))

                print('13 Lap time: %.3f' % ((time.time()-start)))

                quoteData['Hour'] = quoteData.Timestamp.dt.hour
                quoteData['Minute'] = quoteData.Timestamp.dt.minute
                # Adding the ticker
                
            else:

                # Storing the data on the following tickers in a temporary variable.

                temp = pd.DataFrame(tempData, columns= d2.names)
                # Removing all unnecessary variables
                temp = temp.drop(columns=unnecessaryVariables)
                
                print('8 Lap time: %.3f' % ((time.time()-start)))

                temp.loc[:,'ex'] = strList(temp.ex)
                
                print('9 Lap time: %.3f' % ((time.time()-start)))
                
                temp.loc[:,'mode'] = strList(temp['mode'])
                
                print('10 Lap time: %.3f' % ((time.time()-start)))
                
                # Adding the date of the file to the dataframe.
                temp['Date'] = re.split('[._]',file)[1]
                
                print('11 Lap time: %.3f' % ((time.time()-start)))
                
                # Adding a more readable timestamp - TEST IT
                temp['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',file)[1],temp.utcsec))
                
                print('12 Lap time: %.3f' % ((time.time()-start)))
                
                temp['TSRemainder'] = list(map(lambda x: str(x)[11:], temp.utcsec))
                temp['Hour'] = temp.Timestamp.dt.hour
                temp['Minute'] = temp.Timestamp.dt.minute

                # Adding the ticker
                temp['Ticker'] = ticker

                # Adding the new data
                quoteData = pd.concat([quoteData,temp])
            
            # Closing the file after having used it.
            raw_data.close()
    end = time.time()

    print('The extraction time was %.3f seconds.' % (end-start))

    quoteData.loc[:,'price'] = (quoteData.bid + quoteData.ofr) / 2
    quoteData.loc[:,'spread'] = quoteData.ofr - quoteData.bid
    return quoteData


def load_data_v5(dates, tickers, dataNeeded, path, verbose):
    # Measuring the exraction time
    start = time.time()
    
    allFiles = os.listdir(path)

    # Extracting just the dates of each file
    allDates = np.array([re.split("[._]",ele)[1] if ("." in ele ) & ("_" in ele) else 0 for ele in allFiles]).astype(int)

    minDate = np.min(dates)
    maxDate = np.max(dates)
    
    print('1 Lap time: %.3f' % ((time.time()-start)))

    # Locating what files we need.
    index = np.where((minDate <= allDates) & (allDates <= maxDate))
    
    relevantFiles = np.array(allFiles)[index[0]]
    
    print('2 Lap time: %.3f' % ((time.time()-start)))
    
    # Separating the files into trade and quote files.
#     trade = [ele for ele in relevantFiles if 'trade' in ele]
    quote = [ele for ele in relevantFiles if 'quote' in ele]
    
    if (dataNeeded == 'both') | (dataNeeded == 'quotes'):

        # Now to the quote data
        for i,file in enumerate(quote):

#             if (verbose) & (i == 0):
#                 print('### Quote Data ###\n')

            # Reading one file at a time
            raw_data = h5py.File(path+'/'+file,'r')
            dt2 = raw_data['Quotes'].dtype
            print('3 Lap time: %.3f' % ((time.time()-start)))
            
            
            # Store the trade indecies
            QI = raw_data['QuoteIndex']
            
            print('4 Lap time: %.3f' % ((time.time()-start)))
#             if (verbose) & (i==0):
#                 print('The raw H5 quote file contains: ',list(raw_data.keys()),'\n')

            # Extracting just the tickers
            QIC = np.array([ele[0].astype(str).strip() for ele in QI])
            
            print('5 Lap time: %.3f' % ((time.time()-start)))
            pos_start = []
            pos_range = []
            # Lets get data on each ticker for the file processed at the moment
            for j,ticker in enumerate(tickers):
                print(ticker)
                
                tickerInfo = QI[QIC==ticker][0]
                pos_start.append(tickerInfo[1])
                pos_range.append(tickerInfo[2])
                print(tickerInfo)
            print('6 Lap time: %.3f' % ((time.time()-start)))

            # use boolean mask to slice all at once
            selector = zip(pos_start, pos_range) 
            mask = np.zeros(raw_data['Quotes'].shape[0], dtype=bool)
            for (pos_start, pos_range) in selector:
                mask[pos_start : pos_start + pos_range] = True
            tempData = raw_data['Quotes'][mask]
            #print(tempData[:5])
            
            print('7 Lap time: %.3f' % ((time.time()-start)))

            # For first file and first ticker.
            if (i == 0):

                quoteData = pd.DataFrame(tempData, columns= dt2.names)
                # We remove all unnecessary variables
                unnecessaryVariables = ['NationalBBOInd',
                                        'FinraBBOInd',
                                        'FinraQuoteIndicator',
                                        'SequenceNumber',
                                        'FinraAdfMpidIndicator',
                                        'QuoteCancelCorrection',
                                        'SourceQuote',
                                        'RPI',
                                        'ShortSaleRestrictionIndicator',
                                        'LuldBBOIndicator',
                                        'SIPGeneratedMessageIdent',
                                        'NationalBBOLuldIndicator',
                                        'ParticipantTimestamp',
                                        'FinraTimestamp',
                                        'FinraQuoteIndicator',
                                        'SecurityStatusIndicator']

                quoteData = quoteData.drop(columns=unnecessaryVariables)

                print('8 Lap time: %.3f' % ((time.time()-start)))


                quoteData.loc[:,'ex'] = strList(quoteData.ex)
                print('9 Lap time: %.3f' % ((time.time()-start)))

                quoteData.loc[:,'mode'] = strList(quoteData['mode'])
                print('10 Lap time: %.3f' % ((time.time()-start)))

                # Adding the date of the file to the dataframe.
                quoteData['Date'] = re.split('[._]',
                                             file)[1]
                print('11 Lap time: %.3f' % ((time.time()-start)))

                # Adding a more readable timestamp - TEST IT
                dates = pd.to_datetime(quoteData.loc[:,'Date'], format='%Y%m%d', errors='ignore')
                times = pd.to_timedelta(quoteData.loc[:,'utcsec'])
#                 datetimes  = dates + times
#                 datetimes
                quoteData['Timestamp'] = dates + times
#                 quoteData['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',
#                                                                             file)[1],
#                                                                    quoteData.utcsec))
                print('12 Lap time: %.3f' % ((time.time()-start)))

#                 quoteData['TSRemainder'] = list(map(lambda x: str(x)[11:],
#                                                     quoteData.utcsec))

                print('13 Lap time: %.3f' % ((time.time()-start)))

                quoteData['Hour'] = quoteData.Timestamp.dt.hour
                quoteData['Minute'] = quoteData.Timestamp.dt.minute
                # Adding the ticker
                quoteData['Ticker'] = ticker
                
            else:

                # Storing the data on the following tickers in a temporary variable.

                temp = pd.DataFrame(tempData, columns= d2.names)
                # Removing all unnecessary variables
                temp = temp.drop(columns=unnecessaryVariables)
                
                print('8 Lap time: %.3f' % ((time.time()-start)))

                temp.loc[:,'ex'] = strList(temp.ex)
                
                print('9 Lap time: %.3f' % ((time.time()-start)))
                
                temp.loc[:,'mode'] = strList(temp['mode'])
                
                print('10 Lap time: %.3f' % ((time.time()-start)))
                
                # Adding the date of the file to the dataframe.
                temp['Date'] = re.split('[._]',file)[1]
                
                print('11 Lap time: %.3f' % ((time.time()-start)))
                
                # Adding a more readable timestamp - TEST IT
                temp['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',file)[1],temp.utcsec))
                
                print('12 Lap time: %.3f' % ((time.time()-start)))
                
                temp['TSRemainder'] = list(map(lambda x: str(x)[11:], temp.utcsec))
                temp['Hour'] = temp.Timestamp.dt.hour
                temp['Minute'] = temp.Timestamp.dt.minute

                # Adding the ticker
                temp['Ticker'] = ticker

                # Adding the new data
                quoteData = pd.concat([quoteData,temp])
            
            # Closing the file after having used it.
            raw_data.close()
    end = time.time()

    print('The extraction time was %.3f seconds.' % (end-start))

    quoteData.loc[:,'price'] = (quoteData.bid + quoteData.ofr) / 2
    quoteData.loc[:,'spread'] = quoteData.ofr - quoteData.bid
    return quoteData

def load_data_v5_1(dates, tickers, dataNeeded, path, verbose):
    # Measuring the exraction time
    start = time.time()
    
    allFiles = os.listdir(path)
    
    if verbose:
        print(len(allFiles), allFiles[:5], allFiles[-5:])
        print(allFiles[-10:])
    
    # Extracting just the dates of each file
    allDates = np.array([re.split("[._]",ele)[1] if ("." in ele ) & ("_" in ele) else 0 for ele in allFiles]).astype(int)

    minDate = np.min(dates)
    maxDate = np.max(dates)
    
    if verbose:
        print('##### Date range #####\n\nDate, Min: %i\nDate, Max: %i\n'%(minDate,maxDate))
        print('\n1 Lap time: %.3f\n' % ((time.time()-start)))

    # Locating what files we need.
    index = np.where((minDate <= allDates) & (allDates <= maxDate))
    
    relevantFiles = np.array(allFiles)[index[0]]
    
    # Separating the files into trade and quote files.
    trade = [ele for ele in relevantFiles if 'trade' in ele]
    quote = [ele for ele in relevantFiles if 'quote' in ele]
    
    if verbose:
        print('##### Data Extraction begins #####\n')

        if dataNeeded.lower() == 'both':
            print('Both trade and quote data is being extracted..\n')
        else:
            print('%s data is being extracted..\n' % dataNeeded[0:5])
        
        print('\n2 Lap time: %.3f\n' % ((time.time()-start)))
    
    if (dataNeeded == 'both') | (dataNeeded == 'quotes'):

        # Now to the quote data
        for i,file in enumerate(quote):

#             if (verbose) & (i == 0):
#                 print('### Quote Data ###\n')

            # Reading one file at a time
            raw_data = h5py.File(path+'/'+file,'r')
            dt2 = raw_data['Quotes'].dtype
            if verbose:
                print('3 Lap time: %.3f' % ((time.time()-start)))
            
            
            # Store the trade indecies
            QI = raw_data['QuoteIndex']
            
            if verbose:
                print('4 Lap time: %.3f' % ((time.time()-start)))
#             if (verbose) & (i==0):
#                 print('The raw H5 quote file contains: ',list(raw_data.keys()),'\n')

            # Extracting just the tickers
            QIC = np.array([ele[0].astype(str).strip() for ele in QI])
            
            if verbose:
                print('5 Lap time: %.3f' % ((time.time()-start)))
            
            pos_start = []
            pos_range = []
            # Lets get data on each ticker for the file processed at the moment
            for j,ticker in enumerate(tickers):
                
                tickerInfo = QI[QIC==ticker][0]
                pos_start.append(tickerInfo[1])
                pos_range.append(tickerInfo[2])
            
            if verbose:
                print('6 Lap time: %.3f' % ((time.time()-start)))

            # use boolean mask to slice all at once
            selector = zip(pos_start, pos_range) 
            mask = np.zeros(raw_data['Quotes'].shape[0], dtype=bool)
            
            #tickerNp = np.zeros(raw_data['Quotes'].shape[0], dtype=str)
            tickerNp = np.zeros(raw_data['Quotes'].shape[0], dtype='U10')
            
            for t,(pos_start, pos_range) in zip(tickers,selector):
                mask[pos_start : pos_start + pos_range] = True
                tickerNp[pos_start : pos_start + pos_range] = t
                
            tempData = raw_data['Quotes'][mask]
            
            if verbose:
                print('7 Lap time: %.3f' % ((time.time()-start)))

            # For first file and first ticker.
            if (i == 0):

                quoteData = pd.DataFrame(tempData, columns= dt2.names)
                # We remove all unnecessary variables
                unnecessaryVariables = ['NationalBBOInd',
                                        'FinraBBOInd',
                                        'FinraQuoteIndicator',
                                        'SequenceNumber',
                                        'FinraAdfMpidIndicator',
                                        'QuoteCancelCorrection',
                                        'SourceQuote',
                                        'RPI',
                                        'ShortSaleRestrictionIndicator',
                                        'LuldBBOIndicator',
                                        'SIPGeneratedMessageIdent',
                                        'NationalBBOLuldIndicator',
                                        'ParticipantTimestamp',
                                        'FinraTimestamp',
                                        'FinraQuoteIndicator',
                                        'SecurityStatusIndicator']

                quoteData = quoteData.drop(columns=unnecessaryVariables)

                if verbose:
                    print('8 Lap time: %.3f' % ((time.time()-start)))


                quoteData.loc[:,'ex'] = strList(quoteData.ex)
                
                if verbose:
                    print('9 Lap time: %.3f' % ((time.time()-start)))

                quoteData.loc[:,'mode'] = strList(quoteData['mode'])
                
                if verbose:
                    print('10 Lap time: %.3f' % ((time.time()-start)))

                # Adding the date of the file to the dataframe.
                quoteData['Date'] = re.split('[._]',
                                             file)[1]
                if verbose:
                    print('11 Lap time: %.3f' % ((time.time()-start)))

                # Adding a more readable timestamp - TEST IT
                dates = pd.to_datetime(quoteData.loc[:,'Date'], format='%Y%m%d', errors='ignore')
                times = pd.to_timedelta(quoteData.loc[:,'utcsec'])
                quoteData['Timestamp'] = dates + times

                if verbose:
                    print('12 Lap time: %.3f' % ((time.time()-start)))

                #quoteData['Hour'] = quoteData.Timestamp.dt.hour
                #quoteData['Minute'] = quoteData.Timestamp.dt.minute
                # Adding the ticker
                quoteData['Ticker'] = tickerNp[mask]
                
            else:

                # Storing the data on the following tickers in a temporary variable.

                temp = pd.DataFrame(tempData, columns= dt2.names)
                # Removing all unnecessary variables
                temp = temp.drop(columns=unnecessaryVariables)
                
                if verbose:
                    print('8 Lap time: %.3f' % ((time.time()-start)))

                temp.loc[:,'ex'] = strList(temp.ex)
                
                if verbose:
                    print('9 Lap time: %.3f' % ((time.time()-start)))
                
                temp.loc[:,'mode'] = strList(temp['mode'])
                
                if verbose:
                    print('10 Lap time: %.3f' % ((time.time()-start)))
                
                # Adding the date of the file to the dataframe.
                temp['Date'] = re.split('[._]',file)[1]
                
                if verbose:
                    print('11 Lap time: %.3f' % ((time.time()-start)))
                
                # Adding a more readable timestamp - TEST IT
#                 temp['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',file)[1],temp.utcsec))
                dates = pd.to_datetime(temp.loc[:,'Date'], format='%Y%m%d', errors='ignore')
                times = pd.to_timedelta(temp.loc[:,'utcsec'])
                temp['Timestamp'] = dates + times


                if verbose:
                    print('12 Lap time: %.3f' % ((time.time()-start)))
                
#                 temp['TSRemainder'] = list(map(lambda x: str(x)[11:], temp.utcsec))
                #temp['Hour'] = temp.Timestamp.dt.hour
                #temp['Minute'] = temp.Timestamp.dt.minute

                # Adding the ticker
                #temp['Ticker'] = ticker
                quoteData['Ticker'] = tickerNp[mask]

                # Adding the new data
                quoteData = pd.concat([quoteData,temp])
            
            # Closing the file after having used it.
            raw_data.close()
    end = time.time()
    
    quoteData = quoteData.reset_index(drop=True)
    print('The extraction time was %.3f seconds.' % (end-start))

    quoteData.loc[:,'price'] = (quoteData.bid + quoteData.ofr) / 2
    quoteData.loc[:,'spread'] = quoteData.ofr - quoteData.bid
    return quoteData

# v6 is v1 (loop over tickers and read with h5py) with boolean masking instead of read_direct
def load_data_v6(dates, tickers, dataNeeded, path, verbose):
    # Measuring the exraction time
    start = time.time()
    
    allFiles = os.listdir(path)

    # Extracting just the dates of each file
    allDates = np.array([re.split("[._]",ele)[1] if ("." in ele ) & ("_" in ele) else 0 for ele in allFiles]).astype(int)

    minDate = np.min(dates)
    maxDate = np.max(dates)
    
    print('1 Lap time: %.3f' % ((time.time()-start)))

    # Locating what files we need.
    index = np.where((minDate <= allDates) & (allDates <= maxDate))
    
    relevantFiles = np.array(allFiles)[index[0]]
    
    print('2 Lap time: %.3f' % ((time.time()-start)))
    
    # Separating the files into trade and quote files.
#     trade = [ele for ele in relevantFiles if 'trade' in ele]
    quote = [ele for ele in relevantFiles if 'quote' in ele]

    if (dataNeeded == 'both') | (dataNeeded == 'quotes'):

        # Now to the quote data
        for i,file in enumerate(quote):

#             if (verbose) & (i == 0):
#                 print('### Quote Data ###\n')

            # Reading one file at a time
            raw_data = h5py.File(path+'/'+file,'r')
            dt2 = raw_data['Quotes'].dtype
            print('3 Lap time: %.3f' % ((time.time()-start)))
            
            
            # Store the trade indecies
            QI = raw_data['QuoteIndex']
            
            print('4 Lap time: %.3f' % ((time.time()-start)))
#             if (verbose) & (i==0):
#                 print('The raw H5 quote file contains: ',list(raw_data.keys()),'\n')

            # Extracting just the tickers
            QIC = np.array([ele[0].astype(str).strip() for ele in QI])
            
            print('5 Lap time: %.3f' % ((time.time()-start)))
            
            # Lets get data on each ticker for the file processed at the moment
            for j,ticker in enumerate(tickers):
                print(ticker)
                #print(ticker in QIC)
                # Getting the specific ticker information
#                 print(np.where(ticker==QIC))
#                 print(QI[5255])
                #print(QI[QIC==ticker])
                
                tickerInfo = QI[QIC==ticker][0]
                #print(tickerInfo)
                print('6 Lap time: %.3f' % ((time.time()-start)))
                
                
#                 if (verbose) & (i == 0):
#                         print('Ticker Information: ',tickerInfo,'\n')

                # Raw data
#                 tempData = raw_data['Quotes'][list(np.arange(tickerInfo[1],tickerInfo[1]+tickerInfo[2]))]
#                 print(tickerInfo[1],tickerInfo[2])
#                 print('Datatype: ',type(raw_data['Quotes']))
#                 print(raw_data['Quotes'].dtype)
#                 tempData = raw_data['Quotes'][tickerInfo[1]:(tickerInfo[1]+tickerInfo[2])]
#                 tempData = raw_data['Quotes'][403024728:600343]
#                 print(raw_data['Quotes'][[0,1,2,3,4,5]])
#                 print(raw_data['Quotes'][0:5])

                # Read direct
#                 print(raw_data['Quotes'][0])
#                 ll = len(raw_data['Quotes'][0])
#                 print(type(raw_data['Quotes']))
#                 print(type(raw_data['Quotes'][0]))
#                 arr = np.zeros((10,))#,dtype=object,(tickerInfo[2],), dtype='int32'
#                 print(arr)
# #                 print(np.s_[0:10])
#                 raw_data['Quotes'].read_direct(arr,
#                                                np.s_[0:10],#,tickerInfo[1]:tickerInfo[1]+tickerInfo[2]
#                                                np.s_[0:10])#tickerInfo[2]
#                 print(arr[0:5])
        
                ## New proposal
            
#                 store = pd.HDFStore(path+'/'+file, 'r') #.get('Quotes')#'taqquote_20200401.h5'
#                 print(store is None)
#                 meta = store.select('Quotes', start=1, stop=1)
#                 dtDict = [(i,j) for i,j in zip(meta.columns,meta.dtypes)]
#                 dt2 = np.dtype(dtDict)
                
                #testDset = raw_data['Quotes']
                #test = testDset[testDset[0]]
#                 print(dt2)
                arr = np.zeros((tickerInfo[2].astype(int)+1,),dtype=dt2)#,dtype=object,(tickerInfo[2],), dtype='int32'
#                 tickerInfo[1] = 0
#                 tickerInfo[2] = 500000
#                 print(np.s_[0:10])
                #print(int(tickerInfo[1]+tickerInfo[2]+1))
                #print(tickerInfo[1])
                #print(tickerInfo[1]+tickerInfo[2])
                raw_data['Quotes'].read_direct(arr,
                                               np.s_[tickerInfo[1]:int(tickerInfo[1]+tickerInfo[2]+1)],#,tickerInfo[1]:tickerInfo[1]+tickerInfo[2]
                                               np.s_[0:int(tickerInfo[2]+1)])#tickerInfo[2]
#                 print(arr)
#                 tempData = pd.DataFrame(arr,columns=dt2.names)


                print(ticker)
                
                #tickerInfo = QI[QIC==ticker][0]
                #pos_start.append(tickerInfo[1])
                #pos_range.append(tickerInfo[2])
                print(tickerInfo)
                #print('6 Lap time: %.3f' % ((time.time()-start)))

                # use boolean mask to slice all at once 
                mask = np.zeros(raw_data['Quotes'].shape[0], dtype=bool)
                pos_start = tickerInfo[1]
                pos_range = tickerInfo[2]
                mask[pos_start : pos_start + pos_range] = True
                tempData = raw_data['Quotes'][mask]
                #print(tempData[:5])




                print('7 Lap time: %.3f' % ((time.time()-start)))
                
                # For first file and first ticker.
                if (i == 0) & (j == 0):

#                     quoteData = pd.DataFrame(tempData, columns= tempData.dtype.names)
                    quoteData = pd.DataFrame(arr, columns= dt2.names)
                    # We remove all unnecessary variables
                    unnecessaryVariables = ['NationalBBOInd',
                                            'FinraBBOInd',
                                            'FinraQuoteIndicator',
                                            'SequenceNumber',
                                            'FinraAdfMpidIndicator',
                                            'QuoteCancelCorrection',
                                            'SourceQuote',
                                            'RPI',
                                            'ShortSaleRestrictionIndicator',
                                            'LuldBBOIndicator',
                                            'SIPGeneratedMessageIdent',
                                            'NationalBBOLuldIndicator',
                                            'ParticipantTimestamp',
                                            'FinraTimestamp',
                                            'FinraQuoteIndicator',
                                            'SecurityStatusIndicator']

                    quoteData = quoteData.drop(columns=unnecessaryVariables)
                    
                    print('8 Lap time: %.3f' % ((time.time()-start)))
                    
                    
                    quoteData.loc[:,'ex'] = strList(quoteData.ex)
                    print('9 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData.loc[:,'mode'] = strList(quoteData['mode'])
                    print('10 Lap time: %.3f' % ((time.time()-start)))
                    
                    # Adding the date of the file to the dataframe.
                    quoteData['Date'] = re.split('[._]',
                                                 file)[1]
                    print('11 Lap time: %.3f' % ((time.time()-start)))
                    
                    # Adding a more readable timestamp - TEST IT
                    quoteData['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',
                                                                                file)[1],
                                                                       quoteData.utcsec))
                    print('12 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData['TSRemainder'] = list(map(lambda x: str(x)[11:],
                                                        quoteData.utcsec))
                    
                    print('13 Lap time: %.3f' % ((time.time()-start)))
                    
                    quoteData['Hour'] = quoteData.Timestamp.dt.hour
                    quoteData['Minute'] = quoteData.Timestamp.dt.minute
                    # Adding the ticker
                    quoteData['Ticker'] = ticker

                    if (verbose) & (i==0) & (j==0):
                        print('Sneak peak of the data\n\n',quoteData.head())
                    
                    #raw_data.close()
                else:

                    # Storing the data on the following tickers in a temporary variable.

                    temp = pd.DataFrame(arr, columns= dt2.names)
                    # Removing all unnecessary variables
                    temp = temp.drop(columns=unnecessaryVariables)

                    temp.loc[:,'ex'] = strList(temp.ex)
                    temp.loc[:,'mode'] = strList(temp['mode'])

                    # Adding the date of the file to the dataframe.
                    temp['Date'] = re.split('[._]',file)[1]

                    # Adding a more readable timestamp - TEST IT
                    temp['Timestamp'] = pd.to_datetime(formatDate(re.split('[._]',file)[1],temp.utcsec))
                    temp['TSRemainder'] = list(map(lambda x: str(x)[11:], temp.utcsec))
                    temp['Hour'] = temp.Timestamp.dt.hour
                    temp['Minute'] = temp.Timestamp.dt.minute

                    # Adding the ticker
                    temp['Ticker'] = ticker

                    # Adding the new data
                    quoteData = pd.concat([quoteData,temp])

    end = time.time()

    print('The extraction time was %.3f seconds.' % (end-start))

    quoteData.loc[:,'price'] = (quoteData.bid + quoteData.ofr) / 2
    quoteData.loc[:,'spread'] = quoteData.ofr - quoteData.bid
    return quoteData

# Test all
- V1 and v3 seem to not reset index (see output below), but v5 does reset index. V5 is a copy of v2 with boolean masking as slicing
- V2 and V4 are not working for FMNS
- Boolean masking for all tickers (v5) at the same time is fastest (5 sec)
 - V1 is second-fastest with 13 sec
 - then V6 (boolean masking in loop) with 16 sec
 
### In order to start extracting candles within the load_data() function, we need a good way to separate data from each ticker (especially for the non-loop ticker versions)

In [174]:
timer=time.time()
data = load_data(dates, tickers, dataNeeded, path, False)
print(time.time()-timer)
data

24.8605535030365


,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,price,spread
0,14400054219714,P,980.0,5,0.00,0,R,20200501,2020-05-01 04:00:00.054219,714,4,0,GOOG,490.000,-980.00
1,14400054219930,P,1177.0,1,0.00,0,R,20200501,2020-05-01 04:00:00.054219,930,4,0,GOOG,588.500,-1177.00
2,14400054415886,P,1177.0,1,1620.00,1,R,20200501,2020-05-01 04:00:00.054415,886,4,0,GOOG,1398.500,443.00
3,14400054419152,P,1177.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054419,152,4,0,GOOG,1304.750,255.50
4,14400054630302,P,1325.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054630,302,4,0,GOOG,1378.750,107.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757321,71996974108001,K,704.0,1,709.00,20,R,20200501,2020-05-01 19:59:56.974108,001,19,59,TSLA,706.500,5.00
1757322,71997226615556,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.226615,556,19,59,TSLA,706.500,5.00
1757323,71997423917077,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.423917,077,19,59,TSLA,706.500,5.00
1757324,71997741445556,Q,704.0,3,708.39,1,R,20200501,2020-05-01 19:59:57.741445,556,19,59,TSLA,706.195,4.39


In [5]:
data = load_data_v1(dates, tickers, dataNeeded, path, False)
data

1 Lap time: 2.727
2 Lap time: 2.735
3 Lap time: 3.210
4 Lap time: 3.224
5 Lap time: 7.829
GOOG
6 Lap time: 7.836
7 Lap time: 70.235
8 Lap time: 71.552
9 Lap time: 72.750
10 Lap time: 73.792
11 Lap time: 73.809
12 Lap time: 83.536
13 Lap time: 84.578
TSLA
6 Lap time: 84.885
7 Lap time: 267.970
The extraction time was 312.752 seconds.


,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,price,spread
0,14400054219714,P,980.0,5,0.00,0,R,20200501,2020-05-01 04:00:00.054219,714,4,0,GOOG,490.000,-980.00
1,14400054219930,P,1177.0,1,0.00,0,R,20200501,2020-05-01 04:00:00.054219,930,4,0,GOOG,588.500,-1177.00
2,14400054415886,P,1177.0,1,1620.00,1,R,20200501,2020-05-01 04:00:00.054415,886,4,0,GOOG,1398.500,443.00
3,14400054419152,P,1177.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054419,152,4,0,GOOG,1304.750,255.50
4,14400054630302,P,1325.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054630,302,4,0,GOOG,1378.750,107.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757322,71997226615556,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.226615,556,19,59,TSLA,706.500,5.00
1757323,71997423917077,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.423917,077,19,59,TSLA,706.500,5.00
1757324,71997741445556,Q,704.0,3,708.39,1,R,20200501,2020-05-01 19:59:57.741445,556,19,59,TSLA,706.195,4.39
1757325,71999034918242,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:59.034918,242,19,59,TSLA,706.500,5.00


In [6]:
# # not working for FMNS
data = load_data_v2(dates, tickers, dataNeeded, path, False)
data

1 Lap time: 2.452
2 Lap time: 2.458
3 Lap time: 2.989
4 Lap time: 3.006
5 Lap time: 8.520
GOOG
(b'GOOG             ', 357816209, 616180)
TSLA
(b'TSLA             ', 955832818, 1757326)
6 Lap time: 8.527
7 Lap time: 303.433
8 Lap time: 306.131
9 Lap time: 309.320
10 Lap time: 312.162
11 Lap time: 312.203
12 Lap time: 312.976
13 Lap time: 312.978
The extraction time was 314.009 seconds.


,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,Hour,Minute,price,spread
0,14400054219714,P,980.0,5,0.00,0,R,20200501,2020-05-01 04:00:00.054219714,4,0,490.000,-980.00
1,14400054219930,P,1177.0,1,0.00,0,R,20200501,2020-05-01 04:00:00.054219930,4,0,588.500,-1177.00
2,14400054415886,P,1177.0,1,1620.00,1,R,20200501,2020-05-01 04:00:00.054415886,4,0,1398.500,443.00
3,14400054419152,P,1177.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054419152,4,0,1304.750,255.50
4,14400054630302,P,1325.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054630302,4,0,1378.750,107.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373501,71996974108001,K,704.0,1,709.00,20,R,20200501,2020-05-01 19:59:56.974108001,19,59,706.500,5.00
2373502,71997226615556,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.226615556,19,59,706.500,5.00
2373503,71997423917077,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.423917077,19,59,706.500,5.00
2373504,71997741445556,Q,704.0,3,708.39,1,R,20200501,2020-05-01 19:59:57.741445556,19,59,706.195,4.39


In [7]:
data = load_data_v3(dates, tickers, dataNeeded, path, False)
data

1 Lap time: 2.938
2 Lap time: 2.943
3 Lap time: 3.317
4 Lap time: 3.330
5 Lap time: 8.836
GOOG
6 Lap time: 10.777
7 Lap time: 91.459
8 Lap time: 92.172
9 Lap time: 92.174
10 Lap time: 92.174
11 Lap time: 92.187
12 Lap time: 102.865
13 Lap time: 104.058
TSLA
6 Lap time: 104.493
7 Lap time: 346.576
The extraction time was 391.719 seconds.


,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,price,spread
0,14400054219714,P,980.0,5,0.00,0,R,20200501,2020-05-01 04:00:00.054219,714,4,0,GOOG,490.000,-980.00
1,14400054219930,P,1177.0,1,0.00,0,R,20200501,2020-05-01 04:00:00.054219,930,4,0,GOOG,588.500,-1177.00
2,14400054415886,P,1177.0,1,1620.00,1,R,20200501,2020-05-01 04:00:00.054415,886,4,0,GOOG,1398.500,443.00
3,14400054419152,P,1177.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054419,152,4,0,GOOG,1304.750,255.50
4,14400054630302,P,1325.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054630,302,4,0,GOOG,1378.750,107.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757321,71996974108001,K,704.0,1,709.00,20,R,20200501,2020-05-01 19:59:56.974108,001,19,59,TSLA,706.500,5.00
1757322,71997226615556,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.226615,556,19,59,TSLA,706.500,5.00
1757323,71997423917077,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.423917,077,19,59,TSLA,706.500,5.00
1757324,71997741445556,Q,704.0,3,708.39,1,R,20200501,2020-05-01 19:59:57.741445,556,19,59,TSLA,706.195,4.39


In [8]:
# not working for FMNS
data = load_data_v4(dates, tickers, dataNeeded, path, False)
data

1 Lap time: 1.932
2 Lap time: 1.936
3 Lap time: 2.073
4 Lap time: 2.074
5 Lap time: 7.678
GOOG
(b'GOOG             ', 357816209, 616180)
TSLA
(b'TSLA             ', 955832818, 1757326)
6 Lap time: 7.684
7 Lap time: 586.428
8 Lap time: 589.384
9 Lap time: 592.675
10 Lap time: 596.543
11 Lap time: 596.600
12 Lap time: 635.657
13 Lap time: 639.716
The extraction time was 640.778 seconds.


,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,TSRemainder,Hour,Minute,price,spread
0,14400054219714,P,980.0,5,0.00,0,R,20200501,2020-05-01 04:00:00.054219,714,4,0,490.000,-980.00
1,14400054219930,P,1177.0,1,0.00,0,R,20200501,2020-05-01 04:00:00.054219,930,4,0,588.500,-1177.00
2,14400054415886,P,1177.0,1,1620.00,1,R,20200501,2020-05-01 04:00:00.054415,886,4,0,1398.500,443.00
3,14400054419152,P,1177.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054419,152,4,0,1304.750,255.50
4,14400054630302,P,1325.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054630,302,4,0,1378.750,107.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373501,71996974108001,K,704.0,1,709.00,20,R,20200501,2020-05-01 19:59:56.974108,001,19,59,706.500,5.00
2373502,71997226615556,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.226615,556,19,59,706.500,5.00
2373503,71997423917077,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.423917,077,19,59,706.500,5.00
2373504,71997741445556,Q,704.0,3,708.39,1,R,20200501,2020-05-01 19:59:57.741445,556,19,59,706.195,4.39


In [12]:
data = load_data_v5_1(dates, tickers, dataNeeded, path, False)
data

The extraction time was 301.614 seconds.


,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,Hour,Minute,Ticker,price,spread
0,14400054219714,P,980.0,5,0.00,0,R,20200501,2020-05-01 04:00:00.054219714,4,0,GOOG,490.000,-980.00
1,14400054219930,P,1177.0,1,0.00,0,R,20200501,2020-05-01 04:00:00.054219930,4,0,GOOG,588.500,-1177.00
2,14400054415886,P,1177.0,1,1620.00,1,R,20200501,2020-05-01 04:00:00.054415886,4,0,GOOG,1398.500,443.00
3,14400054419152,P,1177.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054419152,4,0,GOOG,1304.750,255.50
4,14400054630302,P,1325.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054630302,4,0,GOOG,1378.750,107.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373501,71996974108001,K,704.0,1,709.00,20,R,20200501,2020-05-01 19:59:56.974108001,19,59,TSLA,706.500,5.00
2373502,71997226615556,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.226615556,19,59,TSLA,706.500,5.00
2373503,71997423917077,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.423917077,19,59,TSLA,706.500,5.00
2373504,71997741445556,Q,704.0,3,708.39,1,R,20200501,2020-05-01 19:59:57.741445556,19,59,TSLA,706.195,4.39


In [13]:
# FMNS
data = load_data_v5_1(dates, tickers, dataNeeded, path, False)
data

ValueError: Length of values does not match length of index

In [10]:
data = load_data_v6(dates, tickers, dataNeeded, path, False)
data

1 Lap time: 2.890
2 Lap time: 2.896
3 Lap time: 3.060
4 Lap time: 3.062
5 Lap time: 8.294
GOOG
6 Lap time: 8.297
GOOG
(b'GOOG             ', 357816209, 616180)
7 Lap time: 134.906
8 Lap time: 135.590
9 Lap time: 136.375
10 Lap time: 137.120
11 Lap time: 137.135
12 Lap time: 148.183
13 Lap time: 149.285
TSLA
6 Lap time: 149.602
TSLA
(b'TSLA             ', 955832818, 1757326)
7 Lap time: 500.954
The extraction time was 539.598 seconds.


,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,price,spread
0,14400054219714,P,980.0,5,0.00,0,R,20200501,2020-05-01 04:00:00.054219,714,4,0,GOOG,490.000,-980.00
1,14400054219930,P,1177.0,1,0.00,0,R,20200501,2020-05-01 04:00:00.054219,930,4,0,GOOG,588.500,-1177.00
2,14400054415886,P,1177.0,1,1620.00,1,R,20200501,2020-05-01 04:00:00.054415,886,4,0,GOOG,1398.500,443.00
3,14400054419152,P,1177.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054419,152,4,0,GOOG,1304.750,255.50
4,14400054630302,P,1325.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054630,302,4,0,GOOG,1378.750,107.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1757322,71997226615556,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.226615,556,19,59,TSLA,706.500,5.00
1757323,71997423917077,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.423917,077,19,59,TSLA,706.500,5.00
1757324,71997741445556,Q,704.0,3,708.39,1,R,20200501,2020-05-01 19:59:57.741445,556,19,59,TSLA,706.195,4.39
1757325,71999034918242,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:59.034918,242,19,59,TSLA,706.500,5.00


In [15]:
%timeit np.zeros(1000000000,dtype=str)

244 ms ± 50.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit np.zeros(1000000000,dtype=np.str)

277 ms ± 52.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit np.zeros(1000000000,dtype=object)

1min 4s ± 5.51 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
n = 1000000 #000
%timeit np.zeros(n, dtype=str)
%timeit np.zeros(n, dtype=np.str)
%timeit np.zeros(n, dtype='U10')
%timeit np.zeros(n, dtype=object)

n = 1000000000
%timeit np.zeros(n, dtype=str)
%timeit np.zeros(n, dtype=np.str)
#%timeit np.zeros(n, dtype=object)

44.1 µs ± 26.2 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
44.5 µs ± 312 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
384 µs ± 503 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
4.42 ms ± 34 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
37.5 ms ± 56.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
37.5 ms ± 63.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
np.zeros(10, dtype='U10')

array(['', '', '', '', '', '', '', '', '', ''], dtype='<U10')

In [156]:
#data_range = np.concatenate([np.arange(i,j+i) for i,j in zip(pos_start,pos_range)])
pos_start, pos_range = [], []
for i in [0,1,2]: 
    pos_start.append(i)
    pos_range.append(3)
    

In [157]:
for i,j in zip(pos_start,pos_range):
    print(i,j)

0 3
1 3
2 3


In [161]:
test_zero = np.arange(10)
#test_zero[np.r_[2:5, 6:9]]

In [162]:
test_zero = np.arange(10)
selector = np.array([[0, 2], [6, 9]])

mask = np.zeros(test_zero.shape[0],dtype=bool)
for (i,j) in selector:
    mask[i:j] = True

out = test_zero[mask]
out

array([0, 1, 6, 7, 8])

In [ ]:
for (i,j) in zip(pos_start,pos_range):
    print(i,i+j)
    mask[i:i+j] = True

In [ ]:
mask

In [163]:
#data_v4 = load_data_v4(dates, tickers, dataNeeded, path, False)

In [ ]:
data_v2 = load_data_v2(dates, tickers, dataNeeded, path, False)

In [164]:
data_v2 = load_data_v2(dates, tickers, dataNeeded, path, False)

1 Lap time: 0.000
2 Lap time: 0.000
3 Lap time: 0.001
4 Lap time: 0.001
5 Lap time: 0.794
GOOG
(b'GOOG             ', 357816209, 616180)
TSLA
(b'TSLA             ', 955832818, 1757326)
6 Lap time: 0.796
[0 1 2 1 2]
[(14400027066425, b'P',  0.  , 0, 100. , 6, b'R', 10401, b'G', b' ', b'', b'', b'C', b'', b'', b'', b'', b'', 14400026882048, 99, b'', b' ')
 (14400027066425, b'P', 67.27, 1, 100. , 6, b'R', 10402, b'G', b' ', b'', b'', b'C', b'', b'', b'', b'', b'', 14400026884608, 99, b'', b' ')
 (14400027066425, b'P', 67.27, 1,  81.1, 1, b'R', 10403, b'G', b' ', b'', b'', b'C', b'', b'', b'', b'', b'', 14400026886144, 99, b'', b' ')
 (14400027088737, b'P', 67.27, 1,  78.2, 1, b'R', 10501, b'G', b' ', b'', b'', b'C', b'', b'', b'', b'', b'', 14400026944512, 99, b'', b' ')
 (14400027088737, b'P', 74.6 , 1,  78.2, 1, b'R', 10503, b'G', b' ', b'', b'', b'C', b'', b'', b'', b'', b'', 14400026946048, 99, b'', b' ')]
[(14400054219714, b'P',  980., 5,    0. , 0, b'R', 2136, b'3', b'', b'', b'', b

In [125]:
#!pip install h5py --user --upgrade

In [111]:
data_v2.head()

,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,Hour,Minute,price,spread
0,14400055186285,P,414.00,1,0.0,0,R,20200501,2020-05-01 04:00:00.055186285,4,0,207.000,-414.00
1,14400055186951,P,518.93,1,0.0,0,R,20200501,2020-05-01 04:00:00.055186951,4,0,259.465,-518.93
2,14400055188470,P,518.93,1,855.0,1,R,20200501,2020-05-01 04:00:00.055188470,4,0,686.965,336.07
3,14400055239554,P,668.00,1,855.0,1,R,20200501,2020-05-01 04:00:00.055239554,4,0,761.500,187.00
4,14400055610520,P,670.00,1,855.0,1,R,20200501,2020-05-01 04:00:00.055610520,4,0,762.500,185.00


In [165]:
data_v2

,utcsec,ex,bid,bidsize,ofr,ofrsize,mode,Date,Timestamp,Hour,Minute,price,spread
0,14400054219714,P,980.0,5,0.00,0,R,20200501,2020-05-01 04:00:00.054219714,4,0,490.000,-980.00
1,14400054219930,P,1177.0,1,0.00,0,R,20200501,2020-05-01 04:00:00.054219930,4,0,588.500,-1177.00
2,14400054415886,P,1177.0,1,1620.00,1,R,20200501,2020-05-01 04:00:00.054415886,4,0,1398.500,443.00
3,14400054419152,P,1177.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054419152,4,0,1304.750,255.50
4,14400054630302,P,1325.0,1,1432.50,2,R,20200501,2020-05-01 04:00:00.054630302,4,0,1378.750,107.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373501,71996974108001,K,704.0,1,709.00,20,R,20200501,2020-05-01 19:59:56.974108001,19,59,706.500,5.00
2373502,71997226615556,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.226615556,19,59,706.500,5.00
2373503,71997423917077,K,704.0,2,709.00,20,R,20200501,2020-05-01 19:59:57.423917077,19,59,706.500,5.00
2373504,71997741445556,Q,704.0,3,708.39,1,R,20200501,2020-05-01 19:59:57.741445556,19,59,706.195,4.39


In [22]:
data_v3 = load_data_v3(dates, tickers, dataNeeded, path, False)

1 Lap time: 2.858
2 Lap time: 2.863
3 Lap time: 3.410
4 Lap time: 3.422
5 Lap time: 8.423
GOOG
6 Lap time: 8.993
7 Lap time: 84.574
8 Lap time: 85.288
9 Lap time: 85.290
10 Lap time: 85.291
11 Lap time: 85.303
12 Lap time: 95.556
13 Lap time: 96.839
TSLA
6 Lap time: 97.135
7 Lap time: 192.666
The extraction time was 208.103 seconds.


In [167]:
data = load_data(dates, tickers, dataNeeded, path, True)

['.ipynb_checkpoints', 'feature_engineering.ipynb', 'Old Versions', 'optimizing_dataload.ipynb', 'optimizing_dataload_FRST.ipynb', 'preprocessing.ipynb', 'Quote_vs_trade_candles.ipynb', 'SpreadInvestigation.ipynb', 'testing.hdf5', 'testing1.hdf5', 'testing_and_comparing_candlesticks.ipynb', 'testing_features.ipynb', 'testing_preprocessing_features_and_labels.ipynb']
1 ['taqquote_20200501.h5'] ['taqquote_20200501.h5']
['taqquote_20200501.h5']
##### Date range #####

Date, Min: 20200501
Date, Max: 20200501

##### Data Extraction begins #####

quote data is being extracted..

### Quote Data ###

The raw H5 quote file contains:  ['QuoteIndex', 'Quotes'] 

Ticker Information:  (b'GOOG             ', 357816209, 616180) 

Sneak peak of the data

            utcsec ex     bid  bidsize     ofr  ofrsize mode      Date  \
0  14400054219714  P   980.0        5     0.0        0    R  20200501   
1  14400054219930  P  1177.0        1     0.0        0    R  20200501   
2  14400054415886  P  1177.0   

## Data Cleaning, Feature Engineering & Pre Processing

# Implemented technical features

A library: https://technical-analysis-library-in-python.readthedocs.io/en/latest/

### Features used in the literature:

* Stochastic K
* Stochastic D
* Slow Stochastic D
* Momentum/difference
* ROC
* Williams % R
* A/D Oscillator
* Disparity 5
* Disparity 10
* Price Oscillator - (detrended)
* Commodity Channel Index
* RSI

Formulas: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=876544

* Moving Average
* Exponential Moving Average
* True Range - (Average)

Formulas: https://www.sciencedirect.com/science/article/pii/S0957417407001819?via%3Dihub

#### Other Technical Features
* Moving Average Convergence Divergence (MACD)

**Non-classical technical features** - **NOT IMPLEMENTED**

* Bid/Ask prices of top of book
* Spread and mid price based on top og book
* Price derivatives

Formulas: https://www.tandfonline.com/doi/full/10.1080/14697688.2015.1032546?instName=UCL+%28University+College+London%29

In [15]:
########### Clean data ###########
    
DATA_SAMPLE = 'full' # or 'stable'

if DATA_SAMPLE == 'stable':
    # P1 is used for keeping data within [9.5, 16]
    cleanedData = HFDataCleaning(['P1','p2','t1','p3'],tradeData,'trade',['q'])
elif DATA_SAMPLE == 'full':
    # P1_2 is used for keeping data within [9, 16.5]
    cleanedData = HFDataCleaning(['P1_2','p2', 'q2', 'p3'],quoteData,'quote',['q'])#'t1',tradeData # q2, quotedate
    
########### Construct Candles ################
# candles = candleCreateNP_vect_final(cleanedData
#                          ,1)

candles = candleCreateNP_vect_final(data = cleanedData,
                                       step = 1,
                                        verbose=False,
                                        fillHoles=True,
                                        sample='stable',
                                        numpied=True)

########### Generate Features ################

n_feature_lags = 0
features = generateFeatures_final(data = candles, 
                                  listOfFeatures = [
                                                    'pastobs',
                                                    'stok',
                                                    'stod',
                                                    'sstod',
                                                    'wilr',
                                                    'roc',
                                                    'rsi',
                                                    'atr',
                                                    'cci',
                                                    'dpo',
                                                    'sma',
                                                    'ema',
                                                    'macd',
                                                    'dis5',
                                                    'dis10',
                                                   ], 
                                   feature_lags = n_feature_lags)

########### Generate Labels ################

n_classes = 3

labels = extract_labels(data = candles, classes = n_classes, group_style = 'equal')

########### Align Data ################

# from imported function (see testing_preprocessing_features_and_labels.ipynb for thorough experimenting with all the cut-offs):    
X, y = align_features_and_labels(candles = candles, 
                                 prediction_horizon = 1, 
                                 features = features, 
                                 n_feature_lags = n_feature_lags, 
                                 n_classes = n_classes, # 5,
                                 safe_burn_in = False, 
                                 data_sample = 'full')